In [1]:
# Ignore the warnings
import warnings
# warnings.filterwarning("always")
# wainfogs.filterwarnings("ignore")

# System related and Data input controls
import os

import re
from typing import *
import missingno as msno

# Web Scraping & Crawling
import requests
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET

import urllib.request
import json
from pandas.io.json import json_normalize

# Data manipulation and visualization
import pandas as pd
import datetime
# pd.options.display.float_format = "{:, 0.2f}".format
# pd.options.display.max_rows = 100
# pd.options.display.max_columns = 20
# pd.set_option("display.max_rows", None)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns

# Modeling algorithms
# (1) General
import statsmodels.api as sm
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats

# (2) Model selection
from sklearn.model_selection import train_test_split

# Evaluation metrics
# for Regression
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# Classification
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sys import executable                                                        
# print(executable)
# C:\ProgramData\anaconda3\python.exe

import sys
# print(sys.path)
# C:\programdata\anaconda3\lib\site-packages

# [Error]
# python3 -m ipykernel install --user
# /C:/ProgramData/anaconda3/python -m pip install request
# !{sys.executable} -m pip install request
# !conda install --yes --prefix {sys.prefix} request

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# GetRepoTradInfoService
    # 기준일자, RP거래일자를 조회하여 
    # 건별거래, 매입증권별잔고금액, 통화별잔고금액을 제공하는 금융위원회_REPO거래정보
    
class GetRepoTradingInfoService():
    
    def __init__(self, 
                 request_url: str, 
                 parameters = None,
                 response = None, 
                 repo_object = None):
        
        self.request_url = request_url
        self.response = response
        self.repo_object = repo_object
            # encoding key = "XaWt3XckF7z5yo3JX%2BMBifN3A6o6cBoIlo3hB0sZznUGKpCHCcOZz4ifqAYfWcRPYLWjJG2dbqEFUXfmr29Xzg%3D%3D"
            # decoding key = "XaWt3XckF7z5yo3JX+MBifN3A6o6cBoIlo3hB0sZznUGKpCHCcOZz4ifqAYfWcRPYLWjJG2dbqEFUXfmr29Xzg=="

            
# requestHTMLWebpage()
    def requestHTMLWebpage(self, baseDate: str):
        
    # 공공데이터포털에서 xml 데이터 요청
        parameters ={
            "serviceKey" : "XaWt3XckF7z5yo3JX+MBifN3A6o6cBoIlo3hB0sZznUGKpCHCcOZz4ifqAYfWcRPYLWjJG2dbqEFUXfmr29Xzg==", 
            "numOfRows" : "1", 
            "pageNo" : "1", 
            "resultType" : "xml", 
            "basDt" : baseDate}
            # 분석에 활용하기 위한 수집 방법
                # 파라미터 딕셔너리의 key값을 변수로 변환하여 class의 method return값을 for문을 통해 날짜별로 생성하여
                # 전역공간에 데이터프레임으로 일별 환매조건부매매채권 데이터 1차원 배열을 병합

        response = requests.get(
            self.request_url, 
            params = parameters)
            # timeout = 0.1)
        self.response = response
            # request_url
                # [통화별잔고금액] http://apis.data.go.kr/1160100/service/GetRepoItemInfoService/getInteRateInfo
                # [건별거래조회] http://apis.data.go.kr/1160100/service/GetRepoTradInfoService/getCaseForTrad
                # [매입증권별 잔고금액 조회] http://apis.data.go.kr/1160100/service/GetRepoTradInfoService/getBuyiSecuBala
        # print(response.content)
        # print(response.text)
                # 요청한 웹페이지 문서가 로드되었는지 확인용
            
    # 데이터 수집
        soup_object = BeautifulSoup(response.content, 
                            "lxml-xml")
        repo_object = soup_object.find_all("item")
        self.repo_object = repo_object
                
        
# Trial & Error : 지속가능성이 없는 방법으로 사용하지 않음
# getInterestRateData()
# REPO 종목정보
    def getInterestRateData(self):
    
    # 데이터 정리
        RepoInterestData: List = []
    
        for i in range(len(self.repo_object)):
            
            baseDate = self.repo_object[i].basDt.string.strip()
            repoID = self.repo_object[i].rpSqno.string.strip()
            
            repoBuySecurities = self.repo_object[i].rpBuyScrtKcd.string.strip()
            repoBuyApplyCurrent = self.repo_object[i].rpBuyAplCurCd.string.strip()
            repoBuyApplyCurrentNM = self.repo_object[i].rpBuyAplCurCdNm.string.strip()
            
            repoDepositTermCode = self.repo_object[i].rdptTermCcd.string.strip()
            repoDepositTermCodeNM = self.repo_object[i].rdptTermCcdNm.string.strip()
            
            repoRemainingExprDcd = self.repo_object[i].rpRmngExprDcd.string.strip()
            repoRemainingExprDcdNm = self.repo_object[i].rpRmngExprDcdNm.string.strip()
            
            purchaseBzTypeCode = self.repo_object[i].purcBzcTcd.string.strip()
            purchaseBzTypeCodeNM = self.repo_object[i].purcBzcTcdNm.string.strip()
            sellerBzTypeCode = self.repo_object[i].slrBzcTcd.string.strip()
            sellerBzTypeCodeNM = self.repo_object[i].slrBzcTcdNm.string.strip()
            
            repoInterest = self.repo_object[i].rpInrt.string.strip()
    
            RepoInterestData = [
                baseDate, repoID, 
                repoBuySecurities, repoBuyApplyCurrent, repoBuyApplyCurrentNM, 
                repoDepositTermCode, repoDepositTermCodeNM, repoRemainingExprDcd, repoRemainingExprDcdNm,
                purchaseBzTypeCode, purchaseBzTypeCodeNM, sellerBzTypeCode, sellerBzTypeCodeNM, repoInterest]
    
            RepoInterestData.append(RepoInterestData)
        
        return RepoInterestData
    

# getTradingData()
# Repo 거래정보
    def getTradingData(self):  
        
        listofItems: List = []
        listofItems = [ [data.text for data in item] for item in self.repo_object ]
        listofItems = list(*listofItems)
          # 이중 리스트에 내포된 <items> tag 내 데이터를 에스터리스크 *를 활용하여 단일 리스트 내 포함되도록 함
        listofItem = list(filter(lambda data: data != "\n", listofItems))
          # <items>
        
        return listofItem

In [ ]:
from tqdm import tqdm
import time
time.sleep(2)

##### 1. 건별 종목정보(금리 등)

getRepoInterest : 환매조건부채권 금리
    
    # baseDate | 기준일자
    # repoID | RP일련번호
    # repoBuySecurities | 기업어음의 매입증권 종류를 관리하는 코드 (통안채, 국채, 은행채 등)
    # repoBuyApplyCurrent | 기업어음의 매입증권 종류를 관리하는 코드 (통안채, 국채, 은행채 등)
    # repoBuyApplyCurrentNM | 환매조건부채권(RP)을 매입 할때 적용되는 통화를 관리하는 코드의 명칭
    # repoDepositTermCode | 상환 기간에 따라 기간을 분류한 코드
    # repoDepositTermCodeNM | 상환 기간에 따라 기간을 분류한 코드의 명칭
    # repoRemainingExprDcd | 기업어음(RP)의 잔존 만기 기간을 구분하는 코드
    # repoRemainingExprDcdNM | 기업어음(RP)의 잔존 만기 기간을 구분하는 코드의 명칭
    # purchaseBzTypeCode | 매수자 업종별 유형을 관리하는 코드
    # purchaseBzTypeCodeNM| 매수자 업종별 유형을 관리하는 코드의 명칭
    # sellerBzTypeCode | 매도자 업종별 유형을 관리하는 코드
    # sellerBzTypeCodeNM | 매도자 업종별 유형을 관리하는 코드의 명칭
    # repoInterest | 환매조건부채권거래 매수자 제공 매입금액에 대한 이율

In [ ]:
getRepoInterest = GetRepoTradingInfoService(
    request_url = "http://apis.data.go.kr/1160100/service/GetRepoItemInfoService/getInteRateInfo"
)
getRepoInterest.requestHTMLWebpage(
    baseDate = "20200304")

In [ ]:
getRepoInterest.getInterestRateData()

['20200304',
 '202003040001',
 '1150',
 'KRW',
 '대한민국 원',
 '1',
 '1영업일',
 '11',
 '1년이하',
 '0202',
 '은행(신탁)',
 '0101',
 '증권사',
 '1.43',
 [...]]

##### 2. 건별 거래조회, 매입증권별 잔고금액 조회, 통화별 잔고금액 조회

(1) getCaseForTrad : RP이율, RP증거금비율, 매입증권매입금액, 매입증권평가금액등을 조회하는 건별거래조회

(2) getBuyiSecuBala : RP국채담보금액, RP금융채담보금액, 지방채담보금액, 담보총합계금액등을 조회

(3) getCurrBala : RP거래일자를 통하여 거래유형코드, RP매입적용통화코드, RP매입적용통화코드명, RP통화잔액등을 조회

In [ ]:
getCaseForTrading = GetRepoTradingInfoService(
    request_url = "http://apis.data.go.kr/1160100/service/GetRepoTradInfoService/getCaseForTrad"
)
getBuyiSecuritiesBalance = GetRepoTradingInfoService(
    request_url = "http://apis.data.go.kr/1160100/service/GetRepoTradInfoService/getBuyiSecuBala"
)
getCurrentBalanceSheet =  GetRepoTradingInfoService(
    request_url = "http://apis.data.go.kr/1160100/service/GetRepoTradInfoService/getCurrBala"
)

In [ ]:
# [Instance] getCaseForTrading
# self.repo_object
getCaseForTrading.requestHTMLWebpage(baseDate = "20200303")
# self.response.text
getCaseForTrading.getTradingData()

# [Instance] getBuyiSecuritiesBalance 
# self.repo_object
getBuyiSecuritiesBalance.requestHTMLWebpage(baseDate = "20200303")
# self.response.text
getBuyiSecuritiesBalance.getTradingData()

# [Instance] getCurrentBalanceSheet
# self.repo_object
getCurrentBalanceSheet.requestHTMLWebpage(baseDate = "20200303")
# self.response.text
getCurrentBalanceSheet.getTradingData()

[<item>
<basDt>20200303</basDt>
<buyScrtBuyAmt>5500000000</buyScrtBuyAmt>
<buyScrtCnt>5800000000</buyScrtCnt>
<buyScrtEvlAmt>5843505800</buyScrtEvlAmt>
<buynShtrFinBzcDcd>0204</buynShtrFinBzcDcd>
<buynShtrFinBzcDcdNm>집합투자</buynShtrFinBzcDcdNm>
<isinCd>KR310502G957</isinCd>
<isinCdNm>중금채(신) 1905이2A-15</isinCdNm>
<rdptTermCcd>2</rdptTermCcd>
<rdptTermCcdNm>2영업일이상 7일미만</rdptTermCcdNm>
<rpBuyAmt>5500000000</rpBuyAmt>
<rpBuyAplCurCd>KRW</rpBuyAplCurCd>
<rpBuyAplCurCdNm>대한민국 원</rpBuyAplCurCdNm>
<rpInrt>1.35</rpInrt>
<rpMrgamRto>105</rpMrgamRto>
<rpOpngDt>20200303</rpOpngDt>
<rpRmngExprDcd>12</rpRmngExprDcd>
<rpRmngExprDcdNm>3년이하</rpRmngExprDcdNm>
<rpSqno>202003030001</rpSqno>
<scrsItmsKcd>1150</scrsItmsKcd>
<scrsItmsKcdNm>은행채</scrsItmsKcdNm>
<slngShtrFinBzcDcd>0204</slngShtrFinBzcDcd>
<slngShtrFinBzcDcdNm>집합투자</slngShtrFinBzcDcdNm>
</item>]
[<item>
<basDt>20200303</basDt>
<etcBondMogAmt>0</etcBondMogAmt>
<etfMogAmt>694159073000</etfMogAmt>
<lcbndMogAmt>348309276509</lcbndMogAmt>
<mogTsumAmt>

['CNY', '위안화', '89360181', '20140829', '02', '']

In [ ]:
CaseForTradingRepo = pd.DataFrame()
col_num = 0

dates = pd.date_range(
    "2020-01-01", 
    "2023-01-01",
    freq = "D")
    # [freq] D, M, Y
    # start_date = pd.to_datetime('2020-01-01')
    # end_date = pd.to_datetime('2023-01-01')
date_list = dates.strftime("%Y%m%d").tolist()

In [ ]:
# self.repo_object
getCaseForTrading.requestHTMLWebpage(baseDate = "20200103")

# self.response.text
getCaseForTrading.getTradingData()

['20200103',
 '9610805222',
 '10000000000',
 '10110250000',
 '0202',
 '은행(신탁)',
 'KR60000148C7',
 '신한은행 22-12-이-3-B',
 '1',
 '1영업일',
 '100000000000',
 'KRW',
 '대한민국 원',
 '1.43',
 '105',
 '20200103',
 '12',
 '3년이하',
 '202001030001',
 '1150',
 '은행채',
 '0101',
 '증권사']

In [ ]:
len(date_list), date_list[0], date_list[-1]

(1097, '20200101', '20230101')

#### 3. 

Repo data : JH Solution

(1) 영업일 및 비영업일 구분하는 try except문

(2) for문에서 dictionary 타입 활용하여 각 시계열 key에 해당하는  환매조건부채권 value값 병합

In [ ]:
for date in date_list[:10]:
    try:
        getCaseForTrading.requestHTMLWebpage(baseDate = date)
        # self.response.text
        print(# "\n column 갯수:", 
              len( getCaseForTrading.getTradingData() ))
        print(bool( getCaseForTrading.getTradingData() ))
        CaseForTradingRepo.loc[col_num, :] = getCaseForTrading.getTradingData()
        
    except:
        print(":비영업일")
        pass

0
False
23
True
:비영업일
23
True
:비영업일
0
False
0
False
23
True
:비영업일
23
True
:비영업일
23
True
:비영업일
23
True
:비영업일
23
True
:비영업일


In [ ]:
tmp_dict = {}

for i, date in enumerate(tqdm(date_list)):
    
    getCaseForTrading.requestHTMLWebpage(baseDate = date)
    
    if bool(getCaseForTrading.getTradingData()) == True:
        # print(len(getCaseForTrading.getTradingData()))
        tmp_dict[i] = getCaseForTrading.getTradingData()
    else:
        # print(len(getCaseForTrading.getTradingData()))
        pass

100%|██████████| 1097/1097 [08:28<00:00,  2.16it/s]


In [ ]:
CaseForTradingRepo = pd.DataFrame( tmp_dict ).transpose()
CaseForTradingRepo.reset_index(drop = True, inplace = True)
CaseForTradingRepo.columns = [
    "기준일자", "매입증권수", "매입증권매입금액", "매입증권평가금액", "매수단기금융업종구분코드", "매수단기금융업종명", "유가증권국제인증고유번호",
    "유가증권국제인증고유번호코드명", "상환기간분류코드", "상환기간분류코드명", 
    "RP매입금액", "RP매입적용통화코드", "RP매입적용통화명", "RP이율", "RP증거금비율", 
    "RP개시일자", "RP잔존만기구분코드", "RP잔존만기", "RP일련번호", "유가증권종목종류코드", # 우선주/보통주
    "유가증권종목명", "매도단기금융업종구분코드", "매도단기금융업종명" 
]
CaseForTradingRepo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   기준일자             680 non-null    object
 1   매입증권수            680 non-null    object
 2   매입증권매입금액         680 non-null    object
 3   매입증권평가금액         680 non-null    object
 4   매수단기금융업종구분코드     680 non-null    object
 5   매수단기금융업종명        680 non-null    object
 6   유가증권국제인증고유번호     680 non-null    object
 7   유가증권국제인증고유번호코드명  680 non-null    object
 8   상환기간분류코드         680 non-null    object
 9   상환기간분류코드명        680 non-null    object
 10  RP매입금액           680 non-null    object
 11  RP매입적용통화코드       680 non-null    object
 12  RP매입적용통화명        680 non-null    object
 13  RP이율             680 non-null    object
 14  RP증거금비율          680 non-null    object
 15  RP개시일자           680 non-null    object
 16  RP잔존만기구분코드       680 non-null    object
 17  RP잔존만기           680 non-null    ob

In [ ]:
CaseForTradingRepo

,기준일자,매입증권수,매입증권매입금액,매입증권평가금액,매수단기금융업종구분코드,매수단기금융업종명,유가증권국제인증고유번호,유가증권국제인증고유번호코드명,상환기간분류코드,상환기간분류코드명,...,RP이율,RP증거금비율,RP개시일자,RP잔존만기구분코드,RP잔존만기,RP일련번호,유가증권종목종류코드,유가증권종목명,매도단기금융업종구분코드,매도단기금융업종명
0,20200110,1000000,1020000000,1283883180,0201,증권사(신탁),KR1035037T34,국고채권 05250-2703(07-3),1,1영업일,...,1.75,108,20200110,14,10년이하,202001100001,1110,국채,0101,증권사
1,20200103,9610805222,10000000000,10110250000,0202,은행(신탁),KR60000148C7,신한은행 22-12-이-3-B,1,1영업일,...,1.43,105,20200103,12,3년이하,202001030001,1150,은행채,0101,증권사
2,20200106,19059592543,20000000000,20031100000,0201,증권사(신탁),KR310508G996,중소기업은행(신) 1909이1.5A-24,1,1영업일,...,1.4,105,20200106,12,3년이하,202001060001,1150,은행채,0701,여신 금융업
3,20200107,1000000,1020000000,1295321460,0201,증권사(신탁),KR1035037T34,국고채권 05250-2703(07-3),1,1영업일,...,1.75,108,20200107,14,10년이하,202001070001,1110,국채,0101,증권사
4,20200108,1000000,1020000000,1289487060,0201,증권사(신탁),KR1035037T34,국고채권 05250-2703(07-3),1,1영업일,...,1.75,108,20200108,14,10년이하,202001080001,1110,국채,0101,증권사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,20221226,3100000000,3750000000,3448946250,0204,집합투자,KR63161419A5,우리금융조건부(상)4(신종)(조건상각/콜/후),2,2영업일이상 7일미만,...,3.25,100,20221226,15,10년초과,202212260001,1160,금융채,0204,집합투자
676,20221227,170000000000,247900000000,181916952800,0301,국내은행,KR103502G933,국고채권 02000-4903(19-2),1,1영업일,...,3.1,100,20221227,15,10년초과,202212270001,1110,국채,0401,보험
677,20221228,500000000000,534100000000,515026220800,0202,은행(신탁),KR103501GB67,국고채권 01125-2406(21-4),1,1영업일,...,3.1,100,20221228,12,3년이하,202212280001,1110,국채,0301,국내은행
678,20221229,170000000000,248700000000,181923055200,0301,국내은행,KR103502G933,국고채권 02000-4903(19-2),1,1영업일,...,3.1,100,20221229,15,10년초과,202212290001,1110,국채,0401,보험


In [ ]:
CaseForTradingRepo.to_csv(
    "/content/drive/MyDrive/archive/CaseForTradingRepoData.csv", 
    index = False,
    encoding = "utf-8"
)
# CaseForTradingRepo.to_excel("./RepoCaseForTradingData.xlsx")

---

In [ ]:
BuyRepoSecuritiesBalance = pd.DataFrame()

for date in date_list[:10]:
    try:
        getBuyiSecuritiesBalance.requestHTMLWebpage(baseDate = date)
        # self.response.text
        print(# "\n column 갯수:", 
              len( getBuyiSecuritiesBalance.getTradingData() ))
        print(bool( getBuyiSecuritiesBalance.getTradingData() ))
        BuyRepoSecuritiesBalance.loc[col_num, :] = getBuyiSecuritiesBalance.getTradingData()
        
    except:
        print(":비영업일")
        pass

0
False
12
True
:비영업일
12
True
:비영업일
0
False
0
False
12
True
:비영업일
12
True
:비영업일
12
True
:비영업일
12
True
:비영업일
12
True
:비영업일


In [ ]:
tmp_dict = {}

for i, date in enumerate(tqdm(date_list)):
    
    getBuyiSecuritiesBalance.requestHTMLWebpage(baseDate = date)
    
    if bool(getBuyiSecuritiesBalance.getTradingData()) == True:
        # print(len(getBuyiSecuritiesBalance.getTradingData()))
        tmp_dict[i] = getBuyiSecuritiesBalance.getTradingData()
    else:
        # print(len(getBuyiSecuritiesBalance.getTradingData()))
        pass

100%|██████████| 1097/1097 [10:01<00:00,  1.82it/s]


In [ ]:
BuyRepoSecuritiesBalance = pd.DataFrame( tmp_dict ).transpose()
BuyRepoSecuritiesBalance.reset_index(drop = True, inplace = True)
BuyRepoSecuritiesBalance.columns = [
    "기준일자", "기타채권담보금액", "ETF담보금액", "지방채담보금액", "RP회사채담보금액",
    "담보총합계금액", "RP회사채담보금액", "RP금융채담보금액", "RP국채담보금액", "RP통안채담보금액", 
    "RP유형코드", "RP유형"
]
BuyRepoSecuritiesBalance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 745 entries, 0 to 744
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   기준일자       745 non-null    int64 
 1   기타채권담보금액   745 non-null    int64 
 2   ETF담보금액    745 non-null    int64 
 3   지방채담보금액    745 non-null    int64 
 4   RP회사채담보금액  745 non-null    int64 
 5   담보총합계금액    745 non-null    int64 
 6   RP회사채담보금액  745 non-null    int64 
 7   RP금융채담보금액  745 non-null    int64 
 8   RP국채담보금액   745 non-null    int64 
 9   RP통안채담보금액  745 non-null    int64 
 10  RP유형코드     745 non-null    int64 
 11  RP유형       745 non-null    object
dtypes: int64(11), object(1)
memory usage: 70.0+ KB


In [ ]:
BuyRepoSecuritiesBalance

,기준일자,기타채권담보금액,ETF담보금액,지방채담보금액,RP회사채담보금액,담보총합계금액,RP회사채담보금액,RP금융채담보금액,RP국채담보금액,RP통안채담보금액,RP유형코드,RP유형
0,20200102,0,773681367000,308589262934,121143155894665,2274373147743,37080147815711,66065555761910,8535866543280,6104941996087,1,기관REPO
1,20200103,0,764050163500,307985906980,118894057064686,2238265733452,37071210314952,64322365536497,8056719789268,6133459620037,1,기관REPO
2,20200106,4957089041,762145704000,276643581308,115669692144394,2370464917986,36263492099076,62317491308585,7416760983662,6257736460736,1,기관REPO
3,20200107,4957870548,755263151500,265736338554,114022695703078,2341148082136,35752705222749,61586627601288,7261508422991,6054749013312,1,기관REPO
4,20200108,4958764384,762029387000,287284482492,115535262416722,2341696467864,36144294887770,62244703713304,7624005236852,6126289477056,1,기관REPO
...,...,...,...,...,...,...,...,...,...,...,...,...
740,20221226,0,1441012350000,387249331313,193136576501302,6655344509213,36547594259324,123538218445915,8163058423987,16404099181550,1,기관REPO
741,20221227,0,1540906785000,375921287777,194655730928261,6573577711936,36196167270187,125401534853721,8163633710705,16403989308935,1,기관REPO
742,20221228,0,1548146855000,378855644625,194851439259849,6510444129195,36266272028456,125553687822155,8573516961752,16020515818666,1,기관REPO
743,20221229,0,1535898355000,370599952574,194704793134209,6412972341205,36167007246044,125706195490261,8042452060156,16469667688969,1,기관REPO


In [ ]:
"""
BuyRepoSecuritiesBalance.to_csv(
    "/content/drive/MyDrive/archive/BuyRepoSecuritiesBalance.csv", 
    index = False,
    encoding = "utf-8")
"""

#### Feature Engineering

* 시간현실반영
* 변수 간 스케일 조정
* 다중공선성 제거

In [ ]:
class TimeSeriesFeatureEngineering():

  def __init__(self,
               trading_repo_path: str,
               bs_repo_path: str):

    self.CaseForTradingRepo = pd.read_csv(
        trading_repo_path, index_col = False, encoding = "utf-8")
    self.BuyRepoSecuritiesBalance = pd.read_csv(
        bs_repo_path, index_col = False, encoding = "utf-8")

  def NonFeatureEngineering(self, date_column, dataframe):

  # 기준일자 컬럼을 date_column변수로 입력받아 날짜형 타입 변환
    if date_column in dataframe.columns:
      dataframe["datetime"] = pd.to_datetime(dataframe[date_column])
      dataframe["DateTime"] = pd.to_datetime(dataframe[date_column])
  # 인덱스 타입이 64비트 정수형이면, DateTime컬럼으로 변경 
    if dataframe.index.dtype == "int64":
      dataframe.set_index("DateTime", inplace = True)

    dataframe_NonFE = dataframe.copy()
    return dataframe_NonFE


  def FeatureEngineering(self, date_column, dataframe):

  # 기준일자 컬럼을 date_column변수로 입력받아 날짜형 타입 변환
    if date_column in dataframe.columns:
      dataframe["datetime"] = pd.to_datetime(dataframe[date_column])
      dataframe["Datetime"] = pd.to_datetime(dataframe[date_column])
  # 인덱스 타입이 64비트 정수형이면, DateTime컬럼으로 변경 
    if dataframe.index.dtype == "int64":
      dataframe.set_index("DateTime", inplace = True)
  # 인덱스 시계열 "Day" 1일 단위로 설정
    dataframe = dataframe.asfreq("D", method = "ffill")

  # 

# if __name__ == '__main__':
    # 클래스를 이용해서 객체를 생성하고 참조값을 변수에 담기


```
# Data Loading
# location = 'https://raw.githubusercontent.com/cheonbi/DataScience/master/Data/Bike_Sharing_Demand_Full.csv'
location = './Data/BikeSharingDemand/Bike_Sharing_Demand_Full.csv'
raw_all = pd.read_csv(location)

# Feature Engineering
raw_fe = feature_engineering(raw_all)
### Reality ###
target = ['count_trend', 'count_seasonal', 'count_Day', 'count_Week', 'count_diff']
raw_feR = feature_engineering_year_duplicated(raw_fe, target)
###############

# Data Split
# Confirm of input and output
Y_colname = ['count']
X_remove = ['datetime', 'DateTime', 'temp_group', 'casual', 'registered']
X_colname = [x for x in raw_fe.columns if x not in Y_colname+X_remove]
X_train_feR, X_test_feR, Y_train_feR, Y_test_feR = datasplit_ts(raw_feR, Y_colname, X_colname, '2012-07-01')
### Reality ###
target = ['count_lag1', 'count_lag2']
X_test_feR = feature_engineering_lag_modified(Y_test_feR, X_test_feR, target)
###############

# Applying Base Model
fit_reg1_feR = sm.OLS(Y_train_feR, X_train_feR).fit()
display(fit_reg1_feR.summary())
pred_tr_reg1_feR = fit_reg1_feR.predict(X_train_feR).values
pred_te_reg1_feR = fit_reg1_feR.predict(X_test_feR).values

# Evaluation
Score_reg1_feR, Resid_tr_reg1_feR, Resid_te_reg1_feR = evaluation_trte(Y_train_feR, pred_tr_reg1_feR,
                                                                   Y_test_feR, pred_te_reg1_feR, graph_on=True)
display(Score_reg1_feR)

# Error Analysis
error_analysis(Resid_tr_reg1_feR, ['Error'], X_train_feR, graph_on=True)
```

In [ ]:
TSFE = TimeSeriesFeatureEngineering(
    trading_repo_path = "/content/drive/MyDrive/archive/CaseForTradingRepoData.csv", 
    bs_repo_path = "/content/drive/MyDrive/archive/BuyRepoSecuritiesBalance.csv"
)

In [ ]:
CaseForTradingRepo["기준일자"] = pd.datetime(CaseForTradingRepo["기준일자"])
# sm.tsa.seasonal_decompose(model = "")
# [input] additive, multicative

,기준일자,매입증권수,매입증권매입금액,매입증권평가금액,매수단기금융업종구분코드,매수단기금융업종명,유가증권국제인증고유번호,유가증권국제인증고유번호코드명,상환기간분류코드,상환기간분류코드명,...,RP이율,RP증거금비율,RP개시일자,RP잔존만기구분코드,RP잔존만기,RP일련번호,유가증권종목종류코드,유가증권종목명,매도단기금융업종구분코드,매도단기금융업종명
0,20200110,1000000,1020000000,1283883180,201,증권사(신탁),KR1035037T34,국고채권 05250-2703(07-3),1,1영업일,...,1.75,108,20200110,14.0,10년이하,202001100001,1110,국채,101,증권사
1,20200103,9610805222,10000000000,10110250000,202,은행(신탁),KR60000148C7,신한은행 22-12-이-3-B,1,1영업일,...,1.43,105,20200103,12.0,3년이하,202001030001,1150,은행채,101,증권사
2,20200106,19059592543,20000000000,20031100000,201,증권사(신탁),KR310508G996,중소기업은행(신) 1909이1.5A-24,1,1영업일,...,1.40,105,20200106,12.0,3년이하,202001060001,1150,은행채,701,여신 금융업
3,20200107,1000000,1020000000,1295321460,201,증권사(신탁),KR1035037T34,국고채권 05250-2703(07-3),1,1영업일,...,1.75,108,20200107,14.0,10년이하,202001070001,1110,국채,101,증권사
4,20200108,1000000,1020000000,1289487060,201,증권사(신탁),KR1035037T34,국고채권 05250-2703(07-3),1,1영업일,...,1.75,108,20200108,14.0,10년이하,202001080001,1110,국채,101,증권사
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,20221226,3100000000,3750000000,3448946250,204,집합투자,KR63161419A5,우리금융조건부(상)4(신종)(조건상각/콜/후),2,2영업일이상 7일미만,...,3.25,100,20221226,15.0,10년초과,202212260001,1160,금융채,204,집합투자
676,20221227,170000000000,247900000000,181916952800,301,국내은행,KR103502G933,국고채권 02000-4903(19-2),1,1영업일,...,3.10,100,20221227,15.0,10년초과,202212270001,1110,국채,401,보험
677,20221228,500000000000,534100000000,515026220800,202,은행(신탁),KR103501GB67,국고채권 01125-2406(21-4),1,1영업일,...,3.10,100,20221228,12.0,3년이하,202212280001,1110,국채,301,국내은행
678,20221229,170000000000,248700000000,181923055200,301,국내은행,KR103502G933,국고채권 02000-4903(19-2),1,1영업일,...,3.10,100,20221229,15.0,10년초과,202212290001,1110,국채,401,보험
